# Facial mapping / landmarks

# 1- Facial mapping (landmarks) with Dlib + python

lien du tutoriel: https://towardsdatascience.com/facial-mapping-landmarks-with-dlib-python-160abcf7d672

#### J'ai pu installer dlib sur windows 10 en utilisant pip install sur Spyder avec cette commande:

#### `pip install https://files.pythonhosted.org/packages/0e/ce/f8a3cff33ac03a8219768f0694c5d703c8e037e6aba2e865f9bae22ed63c/dlib-19.8.1-cp36-cp36m-win_amd64.whl#sha256=794994fa2c54e7776659fddb148363a5556468a6d5d46be8dad311722d54bfcf`

#### `pip install dlib` buggé sur Spyder quand j'exécutais cette commande

In [1]:
import cv2

# if (you have only 1 webcam){ set device = 0} else{ chose your favorite webcam setting device = 1, 2 ,3 ... }
cap=cv2.VideoCapture(0)

while True:
    # Getting our image by webcam and converting it into a gray image scale
    _,image=cap.read()
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    
    #show the grey image
    cv2.imshow("Output",image)
    
    #key to give up the app (it is 'q')
    k=cv2.waitKey(5) & 0xFF
    if k ==ord('q'):
        break;

cv2.destroyAllWindows()
cap.release()


KeyboardInterrupt: 

let's go code an faces detector(HOG) and after detect the landmarks on this detected face

In [2]:

from imutils import face_utils
import dlib
import cv2

# p = our pre-treined model directory, on my case, it's on the same script's diretory.
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

cap = cv2.VideoCapture(0)

while True:
    # Getting out image by webcam 
    _, image = cap.read()
    # Converting the image to gray scale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
    # Get faces into webcam's image
    rects = detector(gray, 0)    
    
    # For each detected face, find the landmark.
    for (i, rect) in enumerate(rects):
        # Make the prediction and transfom it to numpy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
    
        # Draw on our image, all the finded cordinate points (x,y) 
        for (x, y) in shape:
            cv2.circle(image, (x, y), 2, (0, 255, 0), -1)
    
    # Show the image
    cv2.imshow("Output", image)
    
    k = cv2.waitKey(5) & 0xFF
    if k == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()


KeyboardInterrupt: 

# 2-Facial landmarks with dlib, OpenCV, and Python

### (tutoriel plus approfondi)

lien du tutoriel: 
https://www.pyimagesearch.com/2017/04/03/facial-landmarks-dlib-opencv-python/

In [13]:
# import the necessary packages
from collections import OrderedDict
import numpy as np
import cv2

# define a dictionary that maps the indexes of the facial
# landmarks to specific face regions
FACIAL_LANDMARKS_IDXS = OrderedDict([
    ("mouth", (48, 68)),
    ("right_eyebrow", (17, 22)),
    ("left_eyebrow", (22, 27)),
    ("right_eye", (36, 42)),
    ("left_eye", (42, 48)),
    ("nose", (27, 36)),
    ("jaw", (0, 17))
])

#### 1- fonction pour définir le cadre de selection (bounding box) juste pour nous aider pour OpenCV

In [14]:
def rect_to_bb(rect):
    # take a bounding predicted by dlib and convert it
    # to the format (x, y, w, h) as we would normally do
    # with OpenCV
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
 
    # return a tuple of (x, y, w, h)
    return (x, y, w, h)

#### 2-convert this object to a NumPy array, allowing it to “play nicer” with our Python code.

In [15]:
def shape_to_np(shape, dtype="int"):
    # initialize the list of (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)

    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (x, y)-coordinates
    for i in range(0, 68):
        coords[i] = (shape.part(i).x, shape.part(i).y)

    # return the list of (x, y)-coordinates
    return coords

#### Given these two helper functions, we are now ready to detect facial landmarks in images.

writing code here or inside a new file (facial_landmarks.py)

In [16]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# load the input image, resize it, and convert it to grayscale


#image = cv2.imread('img1.jpg')
image= cv2.imread('img2.jpg')

image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # convert dlib's rectangle to a OpenCV-style bounding box
    # [i.e., (x, y, w, h)], then draw the face bounding box
    (x, y, w, h) = face_utils.rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # show the face number
    cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # loop over the (x, y)-coordinates for the facial landmarks
    # and draw them on the image
    for (x, y) in shape:
        cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

# show the output image with the face detections + facial landmarks
cv2.imshow("Output", image)
cv2.waitKey(0)

-1

 # 3- 2nd tutorial + face part detection 

In [18]:
import cv2
from cv2 import *
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import os
import glob
from os.path import basename, splitext



# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# load the input image, resize it, and convert it to grayscale
#image = cv2.imread(data_path + '/'+ dataset + '/'+ img)
image= cv2.imread('img1.jpg')
image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # detect faces in the grayscale image
rects = detector(gray, 1)

    # loop over the face detections
for (i, rect) in enumerate(rects):
        # determine the facial landmarks for the face region, then
        # convert the landmark (x, y)-coordinates to a NumPy array
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)

        # loop over the face parts individually
        for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            # clone the original image so we can draw on it, then
            # display the name of the face part on the image
            clone = image.copy()
            cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 0, 255), 2)

            # loop over the subset of facial landmarks, drawing the
            # specific face part
            for (x, y) in shape[i:j]:
                cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)

            # extract the ROI of the face region as a separate image
            (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
            roi = image[y:y + h, x:x + w]
            try:
                roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
            except ZeroDivisionError:
                print("Plusieurs")


            #Add saving on disk
            #see part4

            cv2.imshow("Output", clone)
            #cv2.imshow("Output", roi)

            k=cv2.waitKey(0)
 

# 4-  3rd tutorial + saving on the disk face parts

In [ ]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# load the input image, resize it, and convert it to grayscale
    
#image = cv2.imread('img1.jpg')
image= cv2.imread('dataset/img2.jpg')

image = imutils.resize(image, width=500)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# detect faces in the grayscale image
rects = detector(gray, 1)

# loop over the face detections
for (i, rect) in enumerate(rects):
    # determine the facial landmarks for the face region, then
    # convert the facial landmark (x, y)-coordinates to a NumPy
    # array
    shape = predictor(gray, rect)
    shape = face_utils.shape_to_np(shape)

    # convert dlib's rectangle to a OpenCV-style bounding box
    # [i.e., (x, y, w, h)], then draw the face bounding box
    (x, y, w, h) = face_utils.rect_to_bb(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # show the face number
    cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # loop over the face parts individually
    for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
        # clone the original image so we can draw on it, then
        # display the name of the face part on the image
        clone = image.copy()
        cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
            0.7, (0, 0, 255), 2)

        # loop over the subset of facial landmarks, drawing the
        # specific face part
        for (x, y) in shape[i:j]:
            cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)

        # extract the ROI of the face region as a separate image
        (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
        roi = image[y:y + h, x:x + w]
        try:
            roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
        except ZeroDivisionError:
            print("Plusieurs")
            
        # show the particular face part
        if not os.path.exists("landmarks/"):
                os.makedirs("landmarks/")

        cv2.imwrite("landmarks/" + name + ".jpg", roi)
  
        cv2.imshow("Output", roi)
        cv2.waitKey(0)

# show the output image with the face detections + facial landmarks
cv2.imshow("Output", image)
cv2.waitKey(0)

# 5-  

4th one + searching all the images in the folders, and devide landmarks into pictures and save them

In [1]:
import cv2
from cv2 import *
from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import os
import glob
from os.path import basename, splitext


# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# load the input image, resize it, and convert it to grayscale


data_path = os.getcwd()+"/dataset"
data_dir_list = os.listdir(data_path)

print(data_dir_list)

for dataset in data_dir_list:
    print(dataset)
    img_list=os.listdir(data_path+'/'+ dataset+'/')
    
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset), img_list)
    
    
    for img in img_list:
        print(img)
    
        #image = cv2.imread('img1.jpg')
        image= cv2.imread(data_path+"/"+dataset+"/"+img)

        image = imutils.resize(image, width=500)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        # detect faces in the grayscale image
        rects = detector(gray, 1)
        
        # clone the original image so we can draw on it, and save the face parts without the draws
        clone = image.copy()

        # loop over the face detections
        for (i, rect) in enumerate(rects):
            # determine the facial landmarks for the face region, then
            # convert the facial landmark (x, y)-coordinates to a NumPy
            # array
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)

            # convert dlib's rectangle to a OpenCV-style bounding box
            # [i.e., (x, y, w, h)], then draw the face bounding box
            (x, y, w, h) = face_utils.rect_to_bb(rect)
            
            cv2.rectangle(clone, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # show the face number
            cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # loop over the face parts individually
            for (name, (k, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
                # display the name of the face part on the image
                cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (0, 0, 255), 2)

                # loop over the subset of facial landmarks, drawing the
                # specific face part
                for (x, y) in shape[k:j]:
                    cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)

                # extract the ROI of the face region as a separate image
                (x, y, w, h) = cv2.boundingRect(np.array([shape[k:j]]))
                roi = image[y:y + h, x:x + w]
                try:
                    roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
                except ZeroDivisionError:
                    print("Plusieurs")

                # show the particular face part
                if not os.path.exists("landmarks/"):
                        os.makedirs("landmarks/")
                
                if not os.path.exists("landmarks/"+dataset+"/"+name+"/"):
                        os.makedirs("landmarks/"+dataset+"/"+name+"/")
                  
                #name because of multiple faces in one image
                img_name=str(i)+img        
                cv2.imwrite("landmarks/" +dataset+"/"+ name+"/"+img_name , roi)

                
        # show the output image with the face detections + facial landmarks
        #cv2.imshow("Output", clone)
        #cv2.waitKey(0)

['dataset1', 'dataset2']
dataset1
Loaded the images of dataset-dataset1
 ['img1.jpg', 'img2.jpg']
img1.jpg
img2.jpg
dataset2
Loaded the images of dataset-dataset2
 ['img3.jpg', 'img4.jpg']
img3.jpg
img4.jpg


# 6- 5th one but on csv